In [1]:
## Import Library

import pandas as pd
import numpy as np
import datetime

## Read Excel

nps_1 = pd.read_excel(
    "C:/Users/yasht/Downloads/Final upload 12 May 2023 (1) (2).xlsx")
nps_2 = pd.read_excel(
    "C:/Users/yasht/Downloads/Final upload 19 May 2023 (1).xlsx")
nps_3 = pd.read_excel(
    "C:/Users/yasht/Downloads/Final upload 26 May 2023 (1).xlsx")
nps_4 = pd.read_excel(
    "C:/Users/yasht/Downloads/Final upload 02 June 2023.xlsx")
nps_5 = pd.read_excel(
    "C:/Users/yasht/Downloads/Final upload 09 June 2023.xlsx")

nps_list = [nps_1, nps_2, nps_3, nps_4, nps_5]

dates = ['12-May-2023', '19-May-2023',
         '26-May-2023', '02-Jun-2023', '09-Jun-2023']


Key KPI - AUM per Subscriber 

Which track the fund total AUM divided by it's subscribers telling the quality of amount brought by an average subscriber. 

Mainly to figure out the flow of the fund.


Running all the function


In [2]:

def clean_data(nps):
    nps.rename(columns={'Pension Fund': "Fund House Name"}, inplace='True')
    nps.rename(columns={'AUM\n(Rs Crs)': 'AUM (Rs Crs)',
                        'Inception\nDate': 'Inception Date',
                        'SCHEME - E Tier-I': 'Scheme Type',
                        'Returns\n 1 Year': 'Returns 1 Year',
                        'Returns \n3 Years': 'Returns 3 Years',
                        'Returns \n5 Years': 'Returns 5 Years',
                        'Returns \n7 Years': 'Returns 7 Years',
                        'Returns \n10 Years': 'Returns 10 Years',
                        'Returns \nInception': 'Returns Inception'}, inplace='True')
    nps.rename(columns={'Pension Fund': "Fund House Name"}, inplace='True')

    nps.loc[0:11, "Scheme Type"] = 'SCHEME - E Tier-I'
    nps.loc[13:24, "Scheme Type"] = 'SCHEME - C Tier-I'
    nps.loc[26:37, "Scheme Type"] = 'SCHEME - G Tier-I'
    nps.loc[39:50, "Scheme Type"] = 'SCHEME - A Tier-I'
    nps.loc[51:62, "Scheme Type"] = 'SCHEME - E Tier-II'
    nps.loc[64:75, "Scheme Type"] = 'SCHEME - C Tier-II'
    nps.loc[77:88, "Scheme Type"] = 'SCHEME - G Tier-II'
    nps.loc[83:87, "Scheme Type"] = 'SCHEME -CG'
    nps.loc[90:94, "Scheme Type"] = 'SCHEME -SG'
    nps.loc[95:100, "Scheme Type"] = 'SCHEME-Corporate SG'
    nps.loc[101:105, "Scheme Type"] = 'SCHEME-Corporate CG'
    nps.loc[106:112, "Scheme Type"] = 'SCHEME- NPS Lite '
    nps.loc[114:117, "Scheme Type"] = 'SCHEME - APY'
    nps.loc[121:131, "Scheme Type"] = 'SCHEME - TTS'

    return nps

def clean_num(nps):
    nps['Returns 1 Year'] = pd.to_numeric(
        nps['Returns 1 Year'], errors='coerce')
    nps['Returns 3 Years'] = pd.to_numeric(
        nps['Returns 3 Years'], errors='coerce')
    nps['Returns 5 Years'] = pd.to_numeric(
        nps['Returns 5 Years'], errors='coerce')
    nps['Returns 7 Years'] = pd.to_numeric(
        nps['Returns 7 Years'], errors='coerce')
    nps['Returns 10 Years'] = pd.to_numeric(
        nps['Returns 10 Years'], errors='coerce')
    nps['Returns Inception'] = pd.to_numeric(
        nps['Returns Inception'], errors='coerce')
    nps['AUM (Rs Crs)'] = round(pd.to_numeric(
        nps['AUM (Rs Crs)'], errors='coerce'), 2)
    nps['Subscribers'] = pd.to_numeric(nps['Subscribers'], errors='coerce')
    nps['Inception Date'] = pd.to_datetime(
        nps['Inception Date'], errors='coerce')
    nps['Returns 1 Year'] = round((nps['Returns 1 Year']*100), 2)
    nps['Returns 3 Years'] = round(nps['Returns 3 Years']*100, 2)
    nps['Returns 5 Years'] = round(nps['Returns 5 Years']*100, 2)
    nps['Returns 7 Years'] = round(nps['Returns 7 Years']*100, 2)
    nps['Returns 10 Years'] = round(nps['Returns 10 Years']*100, 2)
    nps['Returns Inception'] = round(nps['Returns Inception']*100, 2)
    nps['AUM per Subscriber'] = round(
        (nps['AUM (Rs Crs)'] * 10000000) / nps['Subscribers'], 2)
    nps['Tenure of the fund in Yrs'] = round((datetime.datetime.today(
    ) - nps['Inception Date']).astype('timedelta64[D]') / 365.25, 2)
    return nps


def del_row(nps):

    for i in range(len(nps)):
        nps.loc[i, 'Sr.No'] = i
    result = nps[nps['Fund House Name'].isin(['Pension Fund'])]
    nps.drop(result['Sr.No'], axis=0, inplace=True)
    if nps.shape == (133, 12):
        nps.drop([131, 132], inplace=True)
    else:
        nps.drop([131], inplace=True)
    return nps


def calculate_imf(nps):
    nps.loc[nps['AUM (Rs Crs)'] <= 10000, 'Expense Ratio'] = 0.09
    nps.loc[(nps['AUM (Rs Crs)'] > 10000) & (
        nps['AUM (Rs Crs)'] <= 50000), 'Expense Ratio'] = 0.06
    nps.loc[(nps['AUM (Rs Crs)'] > 50000) & (
        nps['AUM (Rs Crs)'] <= 150000), 'Expense Ratio'] = 0.05
    nps.loc[nps['AUM (Rs Crs)'] > 150000, 'Expense Ratio'] = 0.03
    return nps


def rank_1(nps):
    nps_sor = nps.sort_values(
        by=(['Subscribers', 'AUM (Rs Crs)']), ascending=False)
    scheme_tier1 = ['SCHEME - E Tier-I', 'SCHEME - C Tier-I',
                    'SCHEME - A Tier-I', 'SCHEME - G Tier-I']
    tier_1 = nps_sor.loc[nps_sor['Scheme Type'].isin(scheme_tier1)]
    tier_1['Rank_Tier1'] = nps.groupby(
        'Scheme Type')['AUM (Rs Crs)'].rank(ascending=False)

    tier_1 = tier_1[['Scheme Type', 'Fund House Name', 'Inception Date', 'AUM (Rs Crs)',
                     'Subscribers', 'AUM per Subscriber', 'NAV', 'Returns 1 Year', 'Returns 3 Years',
                     'Returns 5 Years', 'Returns 7 Years', 'Returns 10 Years',
                     'Returns Inception', 'Sr.No', 'Expense Ratio',
                     'Rank_Tier1']]
    return tier_1


def rank_all(nps):
    nps_sor = nps.sort_values(
        by=(['Subscribers', 'AUM (Rs Crs)']), ascending=False)
    scheme_name = ['SCHEME - E Tier-I', 'SCHEME - C Tier-I', 'SCHEME - G Tier-I', 'SCHEME - A Tier-I', 'SCHEME - E Tier-II', 'SCHEME - C Tier-II', 'SCHEME - G Tier-II', 'SCHEME -CG', 'SCHEME -SG', 'SCHEME-Corporate SG', 'SCHEME-Corporate CG', 'SCHEME- NPS Lite ', 'SCHEME - APY', 'SCHEME - TTS'
                   ]
    tier_all = nps_sor.loc[nps_sor['Scheme Type'].isin(scheme_name)]
    tier_all['Rank'] = nps.groupby('Scheme Type')[
        'AUM (Rs Crs)'].rank(ascending=False)
    tier_all = tier_all[['Scheme Type', 'Fund House Name', 'Inception Date', 'AUM (Rs Crs)',
                         'Subscribers', 'AUM per Subscriber', 'NAV', 'Returns 1 Year', 'Returns 3 Years',
                         'Returns 5 Years', 'Returns 7 Years', 'Returns 10 Years',
                         'Returns Inception', 'Sr.No', 'Expense Ratio',
                         'Rank']]
    return tier_all


def scheme_report_all(nps):
    scheme_wise = rank_all(nps).groupby(by='Scheme Type').sum(
    ).iloc[:, :3].sort_values(by='AUM (Rs Crs)', ascending=False)
    return scheme_wise


def amc_report_all(nps):
    AMC_summary_all = rank_all(nps).groupby(by='Fund House Name').sum(
    ).iloc[:, :3].sort_values(by='AUM (Rs Crs)', ascending=False)
    return AMC_summary_all


def scheme_report_Tier1(nps):
    scheme_wise = rank_1(nps).groupby(by='Scheme Type').sum(
    ).iloc[:, :3].sort_values(by='AUM (Rs Crs)', ascending=False)
    return scheme_wise


def amc_report_Tier1(nps):
    AMC_summary_Tier1 = rank_1(nps).groupby(by='Fund House Name').sum(
    ).iloc[:, :3].sort_values(by='AUM (Rs Crs)', ascending=False)
    return AMC_summary_Tier1




In [3]:
# Running the code


nps_list = [nps_1, nps_2, nps_3, nps_4, nps_5]


for i, nps_var in enumerate(nps_list):
    nps_var = clean_data(nps_var)
    nps_var = clean_num(nps_var)
    nps_var = del_row(nps_var)
    nps_var = calculate_imf(nps_var)
    nps_list[i] = nps_var



Ranking for all AMC accroding to there category


1. rank_data.xlsx (Rank wise report)


In [4]:
# Ranking Function
nps_rank_list = nps_list.copy()

for i, df in enumerate(nps_rank_list):
    nps_rank = rank_all(df)
    nps_rank_list[i] = nps_rank


nps_status = nps_rank_list[-1]

d = nps_status.groupby(by=['Scheme Type', 'Rank', 'Fund House Name'])
x = d.first()[:]
x.to_excel('rank_data.xlsx', index=True)



C:\Users\yasht\AppData\Local\Temp/ipykernel_24016/15983769.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tier_all['Rank'] = nps.groupby('Scheme Type')[


Tier 1 report 

Downloaded in a excel format


1. AMC_Tier1.xlsx (AMC wise report)


2. Scheme_Tier1.xlsx (Scheme wise report)


In [5]:

for i, nps in enumerate(nps_list):
    var_name = f'scheme_report_Tier1_{i+1}'
    globals()[var_name] = scheme_report_Tier1(nps)



df_scheme_report_Tier1 = [scheme_report_Tier1_1,
                          scheme_report_Tier1_2, scheme_report_Tier1_3, scheme_report_Tier1_4, scheme_report_Tier1_5]


for i, df in enumerate(df_scheme_report_Tier1):
    df['Portfolio Date'] = pd.to_datetime(dates[i], format='%d-%b-%Y')

combined_scheme_Tier1 = pd.concat(
    [scheme_report_Tier1_1, scheme_report_Tier1_2, scheme_report_Tier1_3, scheme_report_Tier1_4, scheme_report_Tier1_5])
combined_scheme_Tier1
combined_scheme_Tier1 = combined_scheme_Tier1.sort_values(
    by=['Scheme Type', 'Portfolio Date', 'AUM (Rs Crs)'])

combined_scheme_Tier1 = pd.concat([scheme_report_Tier1_1, scheme_report_Tier1_2, scheme_report_Tier1_3, scheme_report_Tier1_4, scheme_report_Tier1_5])
combined_scheme_Tier1
combined_scheme_Tier1 = combined_scheme_Tier1.sort_values(by =['Scheme Type', 'Portfolio Date', 'AUM (Rs Crs)'])

combined_scheme_Tier1

combined_scheme_Tier1['Growth Rate Sub'] = round(combined_scheme_Tier1.groupby('Scheme Type')['Subscribers'].pct_change() * 100,2)
combined_scheme_Tier1['Growth Rate AUM per Sub'] = round(combined_scheme_Tier1.groupby('Scheme Type')['AUM per Subscriber'].pct_change() * 100,2)

combined_scheme_Tier1 = combined_scheme_Tier1[['Portfolio Date', 'AUM (Rs Crs)', 'Subscribers', 'Growth Rate Sub', 'AUM per Subscriber','Growth Rate AUM per Sub']]
combined_scheme_Tier1.to_excel('Scheme_Tier1.xlsx', index = True)


for i, nps in enumerate(nps_list):
    var_name = f'amc_report_Tier1_{i+1}'
    globals()[var_name] = amc_report_Tier1(nps)



df_amc_report_Tier1 = [amc_report_Tier1_1,
                       amc_report_Tier1_2, amc_report_Tier1_3, amc_report_Tier1_4, amc_report_Tier1_5]

for i, df in enumerate(df_amc_report_Tier1):
    df['Portfolio Date'] = pd.to_datetime(dates[i], format='%d-%b-%Y')

combined_amc_Tier1 = pd.concat(
    [amc_report_Tier1_1, amc_report_Tier1_2, amc_report_Tier1_3, amc_report_Tier1_4, amc_report_Tier1_5])
combined_amc_Tier1
combined_amc_Tier1 = combined_amc_Tier1.sort_values(
    by=['Fund House Name', 'Portfolio Date'])


combined_amc_Tier1 = pd.concat([amc_report_Tier1_1, amc_report_Tier1_2, amc_report_Tier1_3, amc_report_Tier1_4, amc_report_Tier1_5])
combined_amc_Tier1
combined_amc_Tier1 = combined_amc_Tier1.sort_values(by =['Fund House Name', 'Portfolio Date'])

combined_amc_Tier1['Growth Rate Sub'] = round(combined_amc_Tier1.groupby('Fund House Name')['Subscribers'].pct_change() * 100,2)
combined_amc_Tier1['Growth Rate AUM per Sub'] = round(combined_amc_Tier1.groupby('Fund House Name')['AUM per Subscriber'].pct_change() * 100,2)

combined_amc_Tier1 = combined_amc_Tier1[['Portfolio Date', 'AUM (Rs Crs)', 'Subscribers', 'Growth Rate Sub', 'AUM per Subscriber','Growth Rate AUM per Sub']]
combined_amc_Tier1.to_excel('AMC_Tier1.xlsx', index = True)



C:\Users\yasht\AppData\Local\Temp/ipykernel_24016/15983769.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tier_1['Rank_Tier1'] = nps.groupby(
C:\Users\yasht\AppData\Local\Temp/ipykernel_24016/15983769.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tier_1['Rank_Tier1'] = nps.groupby(


All Report 

Downloaded in a excel format



1. AMC_analysis.xlsx (AMC wise report)


2. scheme_analysis.xlsx (Scheme wise report)

In [6]:

for i, nps in enumerate(nps_list):
    var_name = f'scheme_report_all_{i+1}'
    globals()[var_name] = scheme_report_all(nps)


df_scheme_report_all = [scheme_report_all_1, scheme_report_all_2,
                        scheme_report_all_3, scheme_report_all_4, scheme_report_all_5]


for i, df in enumerate(df_scheme_report_all):
    df['Portfolio Date'] = pd.to_datetime(dates[i], format='%d-%b-%Y')


for i, nps in enumerate(nps_list):
    var_name = f'amc_report_all_{i+1}'
    globals()[var_name] = amc_report_all(nps)


df_amc_report_all = [amc_report_all_1, amc_report_all_2,
                     amc_report_all_3, amc_report_all_4, amc_report_all_5]

for i, df in enumerate(df_amc_report_all):
    df['Portfolio Date'] = pd.to_datetime(dates[i], format='%d-%b-%Y')


combined_amc_all = pd.concat(
    [amc_report_all_1, amc_report_all_2, amc_report_all_3, amc_report_all_4, amc_report_all_5])
combined_amc_all
combined_amc_all = combined_amc_all.sort_values(
    by=['Fund House Name', 'Portfolio Date'])

combined_amc_all

combined_amc_all['Growth Rate Sub'] = round(combined_amc_all.groupby(
    'Fund House Name')['Subscribers'].pct_change() * 100, 2)
combined_amc_all['Growth Rate AUM per Sub'] = round(combined_amc_all.groupby(
    'Fund House Name')['AUM per Subscriber'].pct_change() * 100, 2)


combined_amc_all = combined_amc_all[[
    'Portfolio Date', 'AUM (Rs Crs)', 'Subscribers', 'Growth Rate Sub', 'AUM per Subscriber', 'Growth Rate AUM per Sub']]
combined_amc_all.to_excel('AMC_analysis.xlsx', index=True)


combined_scheme_all = pd.concat([scheme_report_all_1, scheme_report_all_2,
                                scheme_report_all_3, scheme_report_all_4, scheme_report_all_5])
combined_scheme_all
combined_scheme_all = combined_scheme_all.sort_values(
    by=['Scheme Type', 'Portfolio Date', 'AUM (Rs Crs)'])


combined_scheme_all['Growth Rate Sub'] = round(combined_scheme_all.groupby(
    'Scheme Type')['Subscribers'].pct_change() * 100, 2)
combined_scheme_all['Growth Rate AUM per Sub'] = round(combined_scheme_all.groupby(
    'Scheme Type')['AUM per Subscriber'].pct_change() * 100, 2)

combined_scheme_all = combined_scheme_all[[
    'Portfolio Date', 'AUM (Rs Crs)', 'Subscribers', 'Growth Rate Sub', 'AUM per Subscriber', 'Growth Rate AUM per Sub']]
combined_scheme_all.to_excel('scheme_analysis.xlsx', index=True)


C:\Users\yasht\AppData\Local\Temp/ipykernel_24016/15983769.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tier_all['Rank'] = nps.groupby('Scheme Type')[
